In [1]:
# Importing working libraries
import os
import sys
import cv2
import json
import shutil
import random
import librosa
import requests
import datetime
import math as m
import numpy as np
import pandas as pd
from PIL import Image
from io import BytesIO
import librosa.display
import tensorflow as tf
from pathlib import Path
from keras import optimizers
from tensorflow import keras
from google.colab import drive # Comment when working in Visual Studio Code
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from google.colab.patches import cv2_imshow # Comment when working in Visual Studio Code
from tensorflow.keras.models import load_model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Import from my google drive
drive.mount('/content/drive') # Comment when working in Visual Studio Code

# Loaded the main project folder
project_path = "/content/drive/MyDrive/Colab Notebooks/Personal Project/Music Genre Classification"

# Image folder manager
images_original = os.path.join(project_path,"data","images_original")
images_dataset = os.path.join(project_path,"data","images_dataset")
images_dataset_train = os.path.join(images_dataset,"train")
images_dataset_validation = os.path.join(images_dataset,"validation")
images_dataset_test = os.path.join(images_dataset,"test")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# # Generating Spectrograms from Audio

# # Load the audio file
# audio_file = os.path.join(project_path,"data","genres_original","blues","blues.00000.wav")

# # Generate a spectrogram from the audio
# y, sr = librosa.load(audio_file)

# # Generate a spectrogram from the audio
# D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)

# # Display the spectrogram as an image
# plt.figure(figsize=(10, 6))
# librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
# plt.colorbar(format='%+2.0f dB')
# plt.title('Audio Spectrogram')
# plt.show()

In [3]:
# # Splitting images_original to create images_dataset

# # Create dataset, train, validation, and test folders
# os.makedirs(os.path.join(images_dataset, "train"), exist_ok=True)
# os.makedirs(os.path.join(images_dataset, "validation"), exist_ok=True)
# os.makedirs(os.path.join(images_dataset, "test"), exist_ok=True)

# # Proportions
# proportion_train = 0.8
# proportion_validation = 0.1
# proportion_test = 0.1

# # Iterate through each class folder
# for classe in os.listdir(images_original):
#     classe_path = os.path.join(images_original, classe)
#     if os.path.isdir(classe_path):
#         images = os.listdir(classe_path)
#         random.shuffle(images)

#         # Divide the images into training, validation, and test sets
#         num_images = len(images)
#         num_train = int(num_images * proportion_train)
#         num_validation = int(num_images * proportion_validation)

#         train_images = images[:num_train]
#         validation_images = images[num_train:num_train + num_validation]
#         test_images = images[num_train + num_validation:]

#         # Copy the images to the respective folders
#         for image in train_images:
#             src = os.path.join(classe_path, image)
#             dest = os.path.join(images_dataset, "train", classe, image)
#             os.makedirs(os.path.dirname(dest), exist_ok=True)
#             shutil.copy(src, dest)

#         for image in validation_images:
#             src = os.path.join(classe_path, image)
#             dest = os.path.join(images_dataset, "validation", classe, image)
#             os.makedirs(os.path.dirname(dest), exist_ok=True)
#             shutil.copy(src, dest)

#         for image in test_images:
#             src = os.path.join(classe_path, image)
#             dest = os.path.join(images_dataset, "test", classe, image)
#             os.makedirs(os.path.dirname(dest), exist_ok=True)
#             shutil.copy(src, dest)

In [4]:
# Data preparation

batch_size = 1 # Hyperparamètres d'exécution du modèle
# Plus le lot est petit, moins l'estimation du gradient est précise. En résumé : batchsize = le nombre d'exemples vus par le réseau en une seule backpropagation.
# img_height = 288
# img_width = 432
img_height,img_width,img_depth = 288,432,3

# taille suivante associé au choix plus tard de x=16 -> nombre de neurones de la première couche
# n = sqrt(547 770/3x) = 106 -> n² = 11 236
# taille initiale : img_height,img_width,img_depth = 288,432,3
# proportion : prop_h = 288/(288+432) = 288/720 = 0.4 / prop_h = 432/(288+432) = 432/720 = 0.6
# nouvel taille : img_height = n * 2*prop_h = 106*2*0.4 = 84 / img_width = 127 -> 84*127 = 10 668 environ égale à 106² = 11 236
#img_height,img_width,img_depth = 84,127,3

train_data = tf.keras.preprocessing.image_dataset_from_directory(
  images_dataset_train,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# train_data = tf.keras.preprocessing.image_dataset_from_directory(images_dataset_train)

# val_data = tf.keras.preprocessing.image_dataset_from_directory(
#   images_dataset_validation,
#   image_size=(img_height, img_width),
#   batch_size=batch_size)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
  images_dataset_test,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# test_data = tf.keras.preprocessing.image_dataset_from_directory(images_dataset_test)

# class_names = train_data.class_names
# print(class_names)
# print(len(train_data.class_names))
# #print(len(val_data.class_names))
# print(len(test_data.class_names))

#print(train_data)

# print(train_data.shape)

X_train_data = [np.array(X) for X,_ in train_data]
y_train_data = [int(y) for _,y in train_data]

# print(X_train_data.shape)
# print(y_train_data.shape)

X_test_data = [np.array(X) for X,_ in test_data]
y_test_data = [int(y) for _,y in test_data]


Found 799 files belonging to 10 classes.
Found 101 files belonging to 10 classes.


In [5]:
# # print(train_data.shape)
# print(len(train_data))

# print(X_train_data.shape)
# print(y_train_data.shape)

In [6]:
# # Data augmentation

# # Définissez vos paramètres d'augmentation de données
# data_augmentation = tf.keras.Sequential([
#     tf.keras.layers.experimental.preprocessing.RandomFlip("vertical"),
#     #tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
# ])

# # Application de l'augmentation des données (seulement sur le training set)
# print(train_data)
# train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))
# print(train_data)

In [7]:
# # Display of a training set extract

# plt.figure(figsize=(10, 10))
# for images, labels in train_data.take(1):
#   for i in range(batch_size):
#     print(images[i].shape)
#     # ax = plt.subplot(1,batch_size,i+1)
#     # plt.imshow(images[i].numpy().astype("uint8"))
#     # plt.title(class_names[labels[i]])
#     # plt.axis("off")

In [8]:
# import os
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Définir le chemin du dossier contenant les données d'entraînement
# #data_dir = 'chemin/vers/images_dataset_train'

# # Définir la taille des images et le lot (batch size)
# #img_height,img_width,img_depth = 288,432,3
# img_size = (288,432)
# batch_size = 1

# # Utiliser un générateur d'images pour charger les données
# datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# # Charger les données d'entraînement
# train_generator = datagen.flow_from_directory(
#     images_dataset_train,
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='categorical',  # 'categorical' pour les étiquettes one-hot encoded
#     subset='training'  # Spécifier le sous-ensemble comme ensemble d'entraînement
# )

# # Charger les données de validation
# validation_generator = datagen.flow_from_directory(
#     images_dataset_validation,
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset='validation'  # Spécifier le sous-ensemble comme ensemble de validation
# )

# print(train_generator)

# # Extraire les données d'entraînement et d'étiquettes d'entraînement
# X_train_data, y_train_data = next(train_generator)

# # Afficher la forme des données d'entraînement
# print("Shape of X_train_data:", X_train_data.shape)
# print("Shape of y_train_data:", y_train_data.shape)

# print(len(X_train_data))
# print(len(y_train_data))
# print(y_train_data)

# # Extraire les données d'entraînement et d'étiquettes d'entraînement
# X_train_data, y_train_data = next(train_generator)

# # Afficher la forme des données d'entraînement
# print("Shape of X_train_data:", X_train_data.shape)
# print("Shape of y_train_data:", y_train_data.shape)

# print(len(X_train_data))
# print(len(y_train_data))
# print(y_train_data)

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# # Fonction pour extraire les images et les étiquettes
# def extract_images_labels(images, labels):
#     return images, labels

# # Utiliser la méthode map pour appliquer la fonction d'extraction aux éléments de l'ensemble de données
# train_data = train_data.map(extract_images_labels)

# # Diviser les données en X_train_data et y_train_data
# X_train_data, y_train_data = zip(*train_data)

# # Convertir les listes en tableaux numpy (si nécessaire)
# X_train_data = tf.concat(X_train_data, axis=0)
# y_train_data = tf.concat(y_train_data, axis=0)

# # Vérifier les dimensions
# print("Dimensions de X_train_data :", X_train_data.shape)
# print("Dimensions de y_train_data :", y_train_data.shape)

# print(len(train_data))

# # Extraire les données d'entraînement
# for X_train_data, y_train_data in train_data:
#     #print(X_train_data, y_train_data)
#     print(len(X_train_data))
#     print(len(y_train_data))
#     #print(X_train_data)
#     print(y_train_data)
#     # Vous pouvez accéder à X_train_data et y_train_data directement ici
#     break

# X_train_data = []
# y_train_data = []

# for X,y in train_data:
#     X_train_data.append(X)
#     y_train_data.append(y.astype(int))

# import numpy as np

# X_train_data = [np.array(X) for X,_ in train_data]
# y_train_data = [int(y) for _,y in train_data]


# X_train_data, y_train_data = zip(*train_data)

# X_train_data, y_train_data = map(np.array, zip(*train_data))
# X_train_data = X_train_data.astype(int)
# y_train_data = y_train_data.astype(int)


# print(len(X_train_data))
# print(len(y_train_data))
# #print(X_train_data)
# print(y_train_data)

# X_train_data, y_train_data = train_data

# print(len(train_data.class_names))
# #print(len(val_data.class_names))
# print(len(test_data.class_names))





# X_train_data = [np.array(X) for X,_ in train_data]
# y_train_data = [int(y) for _,y in train_data]

# X_test_data = [np.array(X) for X,_ in test_data]
# y_test_data = [int(y) for _,y in test_data]

# --------------------------------------------------------------------------------

# Convertir les listes en tableaux NumPy
X_train_data_array = np.array(X_train_data)
y_train_data_array = np.array(y_train_data)

# Remodeler les données si nécessaire (remplacez 5 par le nombre réel de dimensions)
X_train_data_reshaped = X_train_data_array.reshape((X_train_data_array.shape[0], -1))
y_train_data_reshaped = y_train_data_array.reshape((y_train_data_array.shape[0], -1))

# --------------------------------------------------------------------------------

# Convertir les listes en tableaux NumPy
X_test_data_array = np.array(X_test_data)
y_test_data_array = np.array(y_test_data)

# Remodeler les données si nécessaire (remplacez 5 par le nombre réel de dimensions)
X_test_data_reshaped = X_test_data_array.reshape((X_test_data_array.shape[0], -1))
y_test_data_reshaped = y_test_data_array.reshape((y_test_data_array.shape[0], -1))





In [10]:
print(X_train_data_reshaped.shape)
print(y_train_data_reshaped.shape)

print(X_test_data_reshaped.shape)
print(y_test_data_reshaped.shape)

(799, 373248)
(799, 1)
(101, 373248)
(101, 1)


In [14]:
# Supposons que vous ayez déjà effectué l'ACP comme décrit dans votre code
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler

X_train = X_train_data_reshaped#.T

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

print(X_train.shape)

# Créer l'objet PCA
n = 798
pca = PCA(n_components=n)

# Transformer les données d'apprentissage
X_train_pca = pca.fit_transform(X_train_scaled)

print(X_train_pca.shape)

# # Transformer les données de test
# X_test_pca = pca.transform(X_test_scaled)

# Calculer la variance expliquée par chaque composante principale
explained_variance_ratio = pca.explained_variance_ratio_

# Imprimer la variance expliquée par chaque composante principale
print("Variance expliquée par chaque composante principale:", explained_variance_ratio)

# Imprimer la variance totale expliquée par les deux composantes principales
total_variance_explained = sum(explained_variance_ratio)
print("Variance totale expliquée par les deux composantes principales:", total_variance_explained)
# meilleur var = 42 avec 50 composantes pour l'instant


(799, 373248)
(799, 798)
Variance expliquée par chaque composante principale: [1.93465889e-01 3.76675315e-02 2.26670634e-02 1.20526319e-02
 9.53316409e-03 8.16321932e-03 7.71656306e-03 5.63821476e-03
 5.18934382e-03 5.07750874e-03 4.57497081e-03 4.49765753e-03
 4.09648474e-03 3.96681437e-03 3.71471141e-03 3.59467114e-03
 3.50152212e-03 3.39588523e-03 3.27456975e-03 3.19811585e-03
 3.13695753e-03 3.09890858e-03 3.00254067e-03 2.95068976e-03
 2.91627157e-03 2.84494809e-03 2.83617061e-03 2.78864312e-03
 2.76036048e-03 2.74372008e-03 2.72644893e-03 2.65224581e-03
 2.58625438e-03 2.55734031e-03 2.54886877e-03 2.52381503e-03
 2.51999032e-03 2.46877968e-03 2.45506503e-03 2.44269613e-03
 2.43205903e-03 2.37821601e-03 2.36541033e-03 2.33054953e-03
 2.32078275e-03 2.29592249e-03 2.28278735e-03 2.26103468e-03
 2.24315911e-03 2.22498341e-03 2.21239589e-03 2.18935264e-03
 2.15992844e-03 2.13633548e-03 2.12129857e-03 2.09108833e-03
 2.07551150e-03 2.07080413e-03 2.04211776e-03 2.03893753e-03
 2.0193

In [20]:
# Initialiser le modèle k-NN avec le nombre de voisins (k) souhaité
k = 2
knn_model = KNeighborsClassifier(n_neighbors=k)

# Entraîner le modèle sur les données d'entraînement
knn_model.fit(X_train_pca,y_train_data_reshaped)

# Faire des prédictions sur les données de test
y_pred = knn_model.predict(X_train_pca)

y_train_data_reshaped = y_train_data_reshaped.flatten()
# print(y_train_data_reshaped)
# print(y_pred)

# Calculer la précision du modèle
accuracy = accuracy_score(y_train_data_reshaped,y_pred)
print(f"Précision du modèle : {accuracy}")

# ----------

# # Faire des prédictions sur les données de test
# y_pred = knn_model.predict(X_train_data_reshaped)

# y_train_data_reshaped = y_train_data_reshaped.flatten()
# print(y_train_data_reshaped)
# print(y_pred)

# # Calculer la précision du modèle
# accuracy = accuracy_score(y_train_data_reshaped,y_pred)
# print(f"Précision du modèle : {accuracy}")


Précision du modèle : 0.49687108886107634


In [ ]:
# Model creation

num_classes = 10

# model = tf.keras.Sequential([ # Hyperparamètres des couches (layers)
#     layers.experimental.preprocessing.Rescaling(1./255),
#     layers.Conv2D(128,4, activation='relu'), # Kernel Size # Méthode d'activation des couches cachées
#     layers.MaxPooling2D(),
#     layers.Conv2D(64,4, activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(32,4, activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(16,4, activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Flatten(),
#     layers.Dropout(0.5), # Dropout # Regularization of my model which overfits too much
#     layers.Dense(64,activation='relu'),
#     layers.Dense(num_classes, activation='softmax') # Méthode d'activation du layer final
# ])

# img_height,img_width,img_depth = 288,432,3

# model = tf.keras.Sequential([ # Hyperparamètres des couches (layers)
#     #layers.experimental.preprocessing.Rescaling(1./255),
#     layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height,img_width,img_depth)),
#     layers.Conv2D(128,4, activation='relu'), # Kernel Size # Méthode d'activation des couches cachées
#     layers.MaxPooling2D(),
#     layers.Conv2D(64,4, activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(32,4, activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(16,4, activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Flatten(),
#     layers.Dropout(0.5), # Dropout # Regularization of my model which overfits too much
#     layers.Dense(64,activation='relu'),
#     layers.Dense(num_classes, activation='softmax') # Méthode d'activation du layer final
# ])



#img_height,img_width,img_depth = 288,432,3

model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, img_depth)),
    layers.Flatten(input_shape=(img_height, img_width, img_depth)),
    layers.Dense(1000, activation='relu'),
    #layers.Dropout(0.25), # Rajouter si résultat moyen mais en bonne voie
    layers.Dense(500, activation='relu'),
    layers.Dense(250, activation='relu'),
    layers.Dense(125, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(num_classes, activation='softmax') # num_classes = 10
])

model.compile(optimizer='adam',  # You can choose other optimizers like 'sgd', 'rmsprop', etc.
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' if your labels are one-hot encoded
              metrics=['accuracy'])

# model.compile( # Hyperparamètres de compilation du modèle
#     optimizer='adam', # Optimizer -> descente de gradient stochastique (SGD), Adam, RMSprop, etc.
#     loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False), # Loss utilisée
#     metrics=['accuracy'])


# model.compile(
#     optimizer=optimizers.RMSprop(lr=1e-4),
#     loss='binary_crossentropy',
#     metrics=['acc'])

# Build the model by specifying an input shape
# input_shape = (img_width,img_height,3)  # Adjust this based on your input data shape
# model.build(input_shape)

# Afficher le résumé du modèle, y compris le nombre de paramètres
model.summary()

In [ ]:
# Model training
total_epochs = 30 # Hyperparamètres d'exécution du modèle
#X minutes per epoch -> X hours X minutes for Y training epochs
step_epochs = 2

print("--> Training start time : ", datetime.datetime.now().strftime('%H:%M:%S'))

# Create a loop to train the model in steps
for i in range(0, total_epochs+1, step_epochs):
    start_epoch = i
    end_epoch = min(i + step_epochs, total_epochs)

    print(f"--> Training epochs {start_epoch + 1} to {end_epoch}")

    model.fit(
        train_data,
        validation_data=val_data,
        initial_epoch=start_epoch,
        epochs=end_epoch
    )

    # Save the model after each step
    #template_file = os.path.join(project_path, "models", f"model_{end_epoch}_epochs")
    #model.save(template_file)

print("--> Training end time : ", datetime.datetime.now().strftime('%H:%M:%S'))

In [ ]:
# Restart model training at epoch number X
start_again_epoch_number = 30

# Load model from file
template_file = os.path.join(project_path,"models","model_" + str(start_again_epoch_number) + "_epochs")
model = load_model(template_file)

# Model training
total_epochs = 60 # Hyperparamètres d'exécution du modèle
# X minutes per epoch -> X hours X minutes for Y training epochs
step_epochs = 2

print("--> Training start time : ", datetime.datetime.now().strftime('%H:%M:%S'))

# Create a loop to train the model in steps
for i in range(start_again_epoch_number, total_epochs+1, step_epochs):
    start_epoch = i
    end_epoch = min(i + step_epochs, total_epochs)

    print(f"--> Training epochs {start_epoch + 1} to {end_epoch}")

    model.fit(
        train_data,
        validation_data=val_data,
        initial_epoch=start_epoch,
        epochs=end_epoch
    )

    # Save the model after each step
    #template_file = os.path.join(project_path, "models", f"model_{end_epoch}_epochs")
    #model.save(template_file)

print("--> Training end time : ", datetime.datetime.now().strftime('%H:%M:%S'))

In [ ]:
# # Load model from file
# template_file = os.path.join(project_path,"models","model_26_epochs")
# model = load_model(template_file)

In [ ]:
# # Model evaluation
# evaluation_results = model.evaluate(test_data)

# # Print evaluation results
# print("Validation loss:", evaluation_results[0])
# print("Validation accuracy:", evaluation_results[1])

In [ ]:
# Préparer l'affichage de la courbe d'apprentissage permettant de déterminer l'overfitting

start = 1
end = 30
step = 1

train_losses, val_losses, test_losses = [], [], []
train_accuracies, val_accuracies, test_accuracies = [], [], []

for i in range(start, end+1, step):
    # Load model from file
    template_file = os.path.join(project_path, "models", "model_" + str(i) + "_epochs")
    model = tf.keras.models.load_model(template_file)

    # Evaluation
    train_evaluation = model.evaluate(train_data)
    val_evaluation = model.evaluate(val_data)
    test_evaluation = model.evaluate(test_data)

    # Store evaluation results
    train_losses.append(train_evaluation[0])
    val_losses.append(val_evaluation[0])
    test_losses.append(test_evaluation[0])

    train_accuracies.append(train_evaluation[1])
    val_accuracies.append(val_evaluation[1])
    test_accuracies.append(test_evaluation[1])

In [ ]:
# Afficher la courbe d'apprentissage permettant de déterminer l'overfitting

# Create plots
epochs = list(range(start, end+1, step))

plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'bo-', label='Train Loss')
plt.plot(epochs, val_losses, 'ro-', label='Validation Loss')
plt.plot(epochs, test_losses, 'go-', label='Test Loss')
plt.title('Loss vs. Number of Epochs')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, 'bo-', label='Train Accuracy')
plt.plot(epochs, val_accuracies, 'ro-', label='Validation Accuracy')
plt.plot(epochs, test_accuracies, 'go-', label='Test Accuracy')
plt.title('Accuracy vs. Number of Epochs')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# # Visual Evaluation of Results for Humans

# plt.figure(figsize=(10, 10))
# for images, labels in test_data.take(1):
#   # Prediction
#   predictions = model.predict(images)
#   predicted_classes = predictions.argmax(axis=-1)
#   # Display
#   for i in range(batch_size):
#     plt.figure(figsize=(12, 6))
#     # Display the true class
#     plt.subplot(1, 2, 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title("True class: " + class_names[labels[i]])
#     plt.axis("off")
#     # Display the prediction
#     plt.subplot(1, 2, 2)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title("Prediction: " + class_names[predicted_classes[i]])
#     plt.axis("off")
#   plt.show()